# Live Voltage Output (Moderate)
This example will create a model where neuron voltage is output from the network.

Create a device that extends ```spynnaker.pyNN.external_devices_models.AbstractMulticastControllableDevice```, and adds all the required properties therein (```device_control_partition_id```, ```device_control_key```, ```device_control_uses_payload```, ```device_control_min_value```, ```device_control_max_value```, ```device_control_timesteps_between_sending```, ```device_control_send_type``` and ```device_control_scaling_factor```).  This should have an acceptable voltage range of -50 to 50 and send the value as an integer every 5 timesteps of the simulation, and should use a key of 10.

In [ ]:
from spynnaker.pyNN.external_devices_models import AbstractMulticastControllableDevice
from spynnaker.pyNN.external_devices_models.abstract_multicast_controllable_device import SendType

class TestDevice(AbstractMulticastControllableDevice):
    
    @property
    def device_control_partition_id(self):
        return "Test"
    
    @property
    def device_control_key(self):
        return 0
    
    @property
    def device_control_uses_payload(self):
        return True
    
    @property
    def device_control_min_value(self):
        return -50
    
    @property
    def device_control_max_value(self):
        return 50
    
    @property
    def device_control_timesteps_between_sending(self):
        return 5
    
    @property
    def device_control_send_type(self):
        # The type of the value - one of the SendType values
        return SendType.SEND_TYPE_UINT
    
    @property
    def device_control_scaling_factor(self):
        # The amount to multiply the voltage by before transmission
        return 1.0

Create a translator that extends ```spynnaker.pyNN.external_devices_models.AbstractEthernetTranslator```, and implement the ```translate_control_packet``` method.  This accepts a packet, which has the properties ```key``` and ```payload```.  Print the received key, and the voltage value from the payload.

In [ ]:
from spynnaker.pyNN.external_devices_models import AbstractEthernetTranslator

class Translator(AbstractEthernetTranslator):
    
    def translate_control_packet(self, packet):
        print("Received Key {}: Voltage {}".format(packet.key, packet.payload))

Setup the simulation to run at 1ms timesteps.

In [ ]:
import spynnaker.pyNN as sim

sim.setup(1.0)

Set up a Population of 100 Poisson neurons spiking at a rate of 10Hz.

In [ ]:
source = sim.Population(100, sim.SpikeSourcePoisson(rate=10), label="source")

Set up an External Control Population with 1 neuron using the device and translator created above.

In [ ]:
pop = sim.external_devices.EthernetControlPopulation(
    n_neurons=1,
    model=sim.external_devices.ExternalDeviceLifControl(
        devices=[TestDevice()],
        create_edges=False,
        translator=Translator()))

Connect every neuron in the Poisson source to the control population with a weight of 0.1nA and a delay of 1ms. 

In [ ]:
sim.Projection(source, pop, sim.AllToAllConnector(), sim.StaticSynapse(weight=0.1, delay=1))

Run the simulation for 1000ms, and then end the simulation.  Watch to see the messages produced.

In [ ]:
sim.run(1000)
sim.end()

# Extension 1
Add a live Poisson rate control.  Try to stablise the membrane voltage at 25Hz by altering the rate of the Poisson source based on the live voltage output.

In [ ]:
from spynnaker.pyNN.external_devices_models import AbstractMulticastControllableDevice
from spynnaker.pyNN.external_devices_models.abstract_multicast_controllable_device import SendType


class TestDevice(AbstractMulticastControllableDevice):

    @property
    def device_control_partition_id(self):
        return "Test"

    @property
    def device_control_key(self):
        return 0

    @property
    def device_control_uses_payload(self):
        return True

    @property
    def device_control_min_value(self):
        return -50

    @property
    def device_control_max_value(self):
        return 50

    @property
    def device_control_timesteps_between_sending(self):
        return 5

    @property
    def device_control_send_type(self):
        # The type of the value - one of the SendType values
        return SendType.SEND_TYPE_UINT

    @property
    def device_control_scaling_factor(self):
        # The amount to multiply the voltage by before transmission
        return 1.0

from spynnaker.pyNN.external_devices_models import AbstractEthernetTranslator

class Translator(AbstractEthernetTranslator):

    def __init__(self, connection):
        self._connection = connection
        self._rate = 10
        self._average_voltage = 0

    def translate_control_packet(self, packet):
        print("Received Key {}: Voltage {}".format(packet.key, packet.payload))
        self._average_voltage = ((self._average_voltage * 9) + packet.payload) / 10
        diff = (self._average_voltage - 25) / 50
        new_rate = self._rate - diff
        if new_rate < 0:
            new_rate = 0
        if new_rate != self._rate:
            print("Setting rate to {}".format(new_rate))
            self._rate = new_rate
            self._connection.set_rates("source", [(i, new_rate) for i in range(100)])

import spynnaker.pyNN as sim

connection = sim.external_devices.SpynnakerPoissonControlConnection(local_port=None, poisson_labels=["source"])

sim.setup(1.0)

source = sim.Population(100, sim.SpikeSourcePoisson(rate=10), label="source")
pop = sim.external_devices.EthernetControlPopulation(
    n_neurons=1,
    model=sim.external_devices.ExternalDeviceLifControl(
        devices=[TestDevice()],
        create_edges=False,
        translator=Translator(connection)))

sim.Projection(source, pop, sim.AllToAllConnector(), sim.StaticSynapse(weight=0.1, delay=1))

sim.external_devices.add_poisson_live_rate_control(source, database_notify_port_num=connection.local_port)

sim.run(1000)
sim.end()